In [18]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import mutual_info_classif
from xgboost import XGBClassifier

model = XGBClassifier(max_depth=15,
                      learning_rate=0.05,
                      n_estimators=300,
                      min_child_weight=5,
                      max_delta_step=0,
                      subsample=0.8,
                      colsample_bytree=0.7,
                      reg_alpha=0,
                      reg_lambda=0.4,
                      scale_pos_weight=0.8,
                      silent=True,
                      objective='binary:logistic',
                      eval_metric='error',
                      seed=1440,
                      gamma=0)
#model.fit(train,label5)
# 进行预测
#y_pred = model.predict(test)

#加载数据 预处理
data = pd.read_csv("data/gene_version2020/generate/ExperimentalDatasets/MM/MM-MF.csv",index_col=0)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

sensitivity = []
specificity = []
F1 = []
AUC = []
numList = []
#10折交叉验证
kf = KFold(n_splits=3,shuffle=True)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证
    
    #selectFeatures = IGWeight[:num].index#CFS
    selectFeatures = data.columns

    X_train = trainData.loc[:,selectFeatures]
    X_test = testData.loc[:,selectFeatures]
    #gnb = GaussianNB()
    #predictList = gnb.fit(X_train, Y_train).predict(X_test)
    
    model.fit(X_train,Y_train)
    #print(X_test)
    predictList = model.predict(X_test)
    print(predictList)
    print(Y_test)
    sensi,speci= GM_score(predictList,Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predictList),Y_test))
    try:
        AUC.append(roc_auc_score(np.array(predictList),Y_test))
    except ValueError:
        pass
#standard
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(round(math.sqrt(a*b)*100,1))
print("f1")
print(round(np.nanmean(F1)*100,1))
print("AUC")
print(round(np.nanmean(AUC)*100,1))

c:\users\li\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:21:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1]
[1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 1 0 1 1 0
 0 1 0 0 0 0 1]
[20:21:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




<ipython-input-6-119ee7be13b4>:3: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]
[0 1 1 1 1 1 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1 0 1 1 1 1 0 0 1 0 0 0 1 1
 1 0 1 1 0 1]
[20:21:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




<ipython-input-6-119ee7be13b4>:3: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
c:\users\li\appdata\local\programs\python\python38\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]
[1 1 1 0 1 0 0 0 1 0 1 1 0 1 0 1 1 1 0 1 0 0 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1
 1 1 0 0 1 0]
nan
0.6534531360112755
GM
nan
f1
78.9
AUC
nan


<ipython-input-6-119ee7be13b4>:3: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
<ipython-input-18-9ccf9cf5ecf8>:77: RuntimeWarning: Mean of empty slice
  a = np.nanmean(sensitivity)
<ipython-input-18-9ccf9cf5ecf8>:86: RuntimeWarning: Mean of empty slice
  print(round(np.nanmean(AUC)*100,1))


In [6]:
def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def myknn(train,label,x_test):
    predictList = []
    for j in range(x_test.shape[0]):
        test = x_test.iloc[j,:]
        maxJcd = 0
        prediction = 0
        for i in range(train.shape[0]):
            temp = jaccard_score(train.iloc[i,:],test)
            if(temp > maxJcd):
                prediction = label[i]
                maxJcd = temp
        predictList.append(prediction)
    return predictList

In [4]:
data

,longevity influence,GO:0006006,GO:0006098,GO:0006629,GO:0006695,GO:0006739,GO:0006740,GO:0006749,GO:0009051,GO:0010041,...,GO:2000467,GO:0019369,GO:0019372,GO:0051258,GO:0110076,GO:0002329,GO:0021522,GO:0035195,GO:0060586,GO:0071241
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2136,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2137,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2145,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
